In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests
#Import API key
from config import g_key

In [2]:
# Store cities_csv into a DF
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mahebourg,MU,2022-05-07 17:11:57,-20.4081,57.7000,77.25,88,40,11.50
1,1,Mataura,NZ,2022-05-07 17:11:57,-46.1927,168.8643,58.48,88,84,4.94
2,2,Mar Del Plata,AR,2022-05-07 17:07:35,-38.0023,-57.5575,73.42,53,0,23.02
3,3,Shieli,KZ,2022-05-07 17:11:58,44.1667,66.7500,78.40,32,61,15.99
4,4,Ushuaia,AR,2022-05-07 17:08:59,-54.8000,-68.3000,55.06,47,20,16.11


In [3]:
#Validate that the data columns are either int or float to use gmaps
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use my Google API key
gmaps.configure(api_key=g_key)

In [8]:
#Assign locations to an array of lat and long pairs
locations = city_data_df[['Lat', 'Lng']]
# Assign weights variable to some values
# Get the maximum temperatures
max_temp = city_data_df['Max Temp']

# temps = []
# for temp in max_temp:
#     temps.append(max(temp, 0))

# Assign the figure variable to the gmaps.figure() attribute
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

#Assign the heatmap_layer variable to t he heatmpa_layer attribute and add in the locations
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, 
                                 max_intensity = 300, point_radius=4)

#Add heatmap layer
fig.add_layer(heat_layer)

#Call the fig to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap for % humidity

locations= city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap for % cloudiness

locations= city_data_df[['Lat','Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap for Wind Speed (mph)

locations= city_data_df[['Lat','Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
# Ask customer to add a min and max temperature value
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [48]:
# Filter dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)].dropna()

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mahebourg,MU,2022-05-07 17:11:57,-20.4081,57.7000,77.25,88,40,11.50
3,3,Shieli,KZ,2022-05-07 17:11:58,44.1667,66.7500,78.40,32,61,15.99
10,10,Bongandanga,CD,2022-05-07 17:12:02,1.5000,21.0500,77.65,74,94,3.69
12,12,Cabo San Lucas,MX,2022-05-07 17:10:10,22.8909,-109.9124,87.62,47,0,4.61
17,17,Kapaa,US,2022-05-07 17:12:06,22.0752,-159.3190,75.52,82,100,18.41


In [49]:
# DF called hotel_df to store hotel names along with city, country, max temp and coordinates
hotel_df = preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name']=""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mahebourg,MU,77.25,-20.4081,57.7000,
3,Shieli,KZ,78.40,44.1667,66.7500,
10,Bongandanga,CD,77.65,1.5000,21.0500,
12,Cabo San Lucas,MX,87.62,22.8909,-109.9124,
17,Kapaa,US,75.52,22.0752,-159.3190,


In [50]:
# Set the parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": ""}
#Iterate through the DF

for index, row in hotel_df.iterrows():
    #Get lat and long
    lat = row['Lat']
    lng = row['Lng']
    
    #Add the lat and long to location key for the params dictionary
   
    params['location']= f"{lat},{lng}"
    
    #Use the sear term: lodging and lat and long
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    #Grab the first hotel from the results and store the name

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print(f"Hotel not found in...skipping")


Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping
Hotel not found in...skipping


In [46]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Cabo San Lucas,MX,87.62,22.8909,-109.9124,Hotel Tesoro Los Cabos
20,Bondo,CD,80.17,3.8146,23.6866,
29,Butaritari,KI,81.28,3.0707,172.7902,Isles Sunset Lodge
36,Touros,BR,80.60,-5.1989,-35.4608,INN NEW HORIZON
41,Allapalli,IN,92.43,19.4167,80.0667,"Hotel IK Residency, GUEST HOUSE AC NON AC"


In [53]:
# Info box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
""" 

#Store the DF Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add heatmap of temperature for the vacation spots
locations = hotel_df[['Lat','Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))